### xPath

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news

Для парсинга использовать xpath. Структура данных должна содержать:

    название источника,
    
    наименование новости,
    
    ссылку на новость,
    
    дата публикации

2)Сложить все новости в БД

In [21]:
from pymongo import MongoClient
from pprint import pprint
from lxml import html
import requests
import itertools

In [22]:
client = MongoClient('localhost',27017)

In [23]:
db = client['news']

In [24]:
news_mail_ru  =db.mail

In [25]:
news_lenta_ru = db.lenta

In [26]:
news_yandex_ru = db.yandex

# news.mail.ru

In [27]:
def requests_to_link(main_link, headers):
    response = requests.get(main_link, headers = headers)
    dom = html.fromstring(response.text)
    
    link_1 = dom.xpath('//div[@class="cols__wrapper"]//a/@href')  
    
    link_main_js = dom.xpath('//div[@class = "daynews__item daynews__item_big"]//a/@href')
    
    link_main_other = dom.xpath('//div[@class="daynews__item"]//a/@href')
    
    link_block_after_main = dom.xpath('//div[@class="js-module"]//ul//a[@class = "list__text"]/@href')
    
    return link_main_other + link_main_js+link_block_after_main , link_1

In [300]:
main_link = 'https://news.mail.ru'

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

In [302]:
# Сохраняем список с найденными ссылками 
link_list_1, link_list_2 = requests_to_link(main_link,headers)

In [317]:
count =0
for news_link_1 in link_list_2:
    
    if 'https' in news_link_1:
        response = requests.get(news_link_1, headers = headers)
        
        
        dom = html.fromstring(response.text)
        time = dom.xpath('//div[@class="cols__inner"]//span[@class = "note__text breadcrumbs__text js-ago"]/text()')
        news_name = dom.xpath('//div[@class="hdr__wrapper"]//h1[@class="hdr__inner"]/text()')      
        source = dom.xpath('//div[@class = "breadcrumbs breadcrumbs_article js-ago-wrapper"]//span[@class = "link__text"]/text()')
        try:
            news_mail_ru.insert_one({'_id':''.join(news_name),
                                'time':time,
                                'source':source,
                                'link':news_link_1})
        except:
            continue
            
        
        
for  news_link_2 in link_list_1:
    
    if 'https' in news_link_2:
        response = requests.get(news_link_2, headers = headers)
        dom = html.fromstring(response.text)
        
        
        time = dom.xpath('//div[@class="cols__inner"]//span[@class = "note__text breadcrumbs__text js-ago"]/text()')
        news_name = dom.xpath('//div[@class="hdr__wrapper"]//h1[@class="hdr__inner"]/text()')      
        source = dom.xpath('//div[@class = "breadcrumbs breadcrumbs_article js-ago-wrapper"]//span[@class = "link__text"]/text()')
        try:
            news_mail_ru.insert_one({'_id':''.join(news_name),
                                'time':time,
                                'source':source,
                               'link':news_link_1})
        except:
            continue
        
    else:
        response = requests.get(main_link + news_link_2, headers = headers)
        dom = html.fromstring(response.text)
        
        
        time = dom.xpath('//div[@class="cols__inner"]//span[@class = "note__text breadcrumbs__text js-ago"]/text()')
        news_name = dom.xpath('//div[@class="hdr__wrapper"]//h1[@class="hdr__inner"]/text()')      
        source = dom.xpath('//div[@class = "breadcrumbs breadcrumbs_article js-ago-wrapper"]//span[@class = "link__text"]/text()')
        try:
            news_mail_ru.insert_one({'_id':''.join(news_name),
                                'time':time,
                                'source':source,
                                'link':news_link_1})
        except:
            continue


In [318]:
for i in news_mail_ru.find({}):
    pprint(i)

{'_id': 'Число новых случаев COVID-19 сократилось за месяц на 31 процент',
 'link': 'https://news.mail.ru/society/42491953/',
 'source': ['Официальный портал Мэра и Правительства Москвы'],
 'time': ['11:34 (мск)']}
{'_id': 'Москвичей предупредили о грозе, граде и сильном ветре в среду',
 'link': 'https://news.mail.ru/society/42489347/',
 'source': ['Интерфакс'],
 'time': ['11:58 (мск)']}
{'_id': 'Посещаемость в ресторанах и кафе Москвы в июле восстановили на 40% от '
        'марта',
 'link': 'https://news.mail.ru/society/42490557/',
 'source': ['ТАСС'],
 'time': ['11:07 (мск)']}
{'_id': '',
 'link': 'https://news.mail.ru/society/42497607/',
 'source': [],
 'time': []}
{'_id': 'Московским театрам разрешили открыться с 1 августа',
 'link': 'https://news.mail.ru/society/42497607/',
 'source': ['ТАСС'],
 'time': ['16:39 (мск)']}
{'_id': 'Кинотеатры в Москве возобновят работу с 1 августа',
 'link': 'https://news.mail.ru/society/42495474/',
 'source': ['ТАСС'],
 'time': ['14:46 (мск)']}
{'_

# lenta.ru

In [305]:
def requests_to_link(main_link, headers):
    response = requests.get(main_link, headers = headers)
    dom = html.fromstring(response.text)
    
    link_main = dom.xpath('//div[@class = "span8 js-main__content"]//h2/a/@href') 
    link_item = dom.xpath('//div[@class = "item"]/a/@href')
    link_h3 = dom.xpath('//div[@class = "span8 js-main__content"]//h3/a/@href')
    link_other = dom.xpath('//div[@class = "b-feature__header"]/a/@href')
    
    

    
    return link_main+link_item+link_h3+link_other

In [306]:
main_link_lenta = 'https://lenta.ru/'


In [307]:
link_lenta = requests_to_link(main_link_lenta,headers)

In [308]:
for i in range(len(link_lenta)):
    if 'http' not in link_lenta[i]:
        link_lenta[i] = main_link_lenta + link_lenta[i]

In [309]:
for news_lents in link_lenta:
    
    if '/lenta.ru' in news_lents:
        response = requests.get(news_lents, headers = headers) 
        
        dom = html.fromstring(response.text)
        time = dom.xpath('//div[@class = "b-topic__info"]/time/text()')
        news_name = dom.xpath('//h1[@class = "b-topic__title"]/text()')
        source = 'Лента Ру'
        
        
            
        if not  news_name and not time :
            continue
        
        if not time:
            time = None
        try:
            news_lenta_ru.insert_one({'_id':''.join(news_name),
                                'time':time,
                                'source':source,
                                'link':news_lents})
        except:
            continue
    

In [310]:
for i in news_lenta_ru.find({}):
    pprint(i)

{'_id': ObjectId('5f05c11d60dbad26b449d129'),
 'link': 'https://lenta.ru//news/2020/07/08/comment/',
 'name_news': ['Чехия отказалась комментировать обвинения против советника '
               'главы «Роскосмоса»'],
 'source': 'Лента Ру',
 'time': [' 12:42,  8 июля 2020']}
{'_id': ObjectId('5f05c11d60dbad26b449d12a'),
 'link': 'https://lenta.ru//news/2020/07/08/biden_in_ussr/',
 'name_news': ['Найдено свидетельство первого визита Байдена в\xa0СССР'],
 'source': 'Лента Ру',
 'time': [' 12:46,  8 июля 2020']}
{'_id': ObjectId('5f05c11d60dbad26b449d12b'),
 'link': 'https://lenta.ru//news/2020/07/08/kimk/',
 'name_news': ['Кожаный наряд Ким Кардашьян с\xa0глубоким декольте сравнили '
               'с\xa0пакетом для мусора'],
 'source': 'Лента Ру',
 'time': [' 12:39,  8 июля 2020']}
{'_id': ObjectId('5f05c11d60dbad26b449d12c'),
 'link': 'https://lenta.ru//news/2020/07/08/domoi/',
 'name_news': ['Украина выдворила двух задержанных россиян'],
 'source': 'Лента Ру',
 'time': [' 12:34,  8 июля

# Яндекс Новости 

In [29]:
mainlink_ya = 'https://yandex.ru'

In [30]:
def requests_to_link_ya(main_link, headers):
    response = requests.get(main_link+'/news', headers = headers)
    dom = html.fromstring(response.text)
    
    news_link = dom.xpath('//div[@class = "page-content"]//h2/a/@href')
    name_news = dom.xpath('//div[@class="story__topic"]/h2/a/text()')
    time =[i[-5:] for i in dom.xpath('//div[@class = "story__date"]/text()')]
    source = [i[:-5] for i in dom.xpath('//div[@class = "story__date"]/text()')]
    
    return (news_link,source,name_news,time)


In [31]:
# Делаем распоковку в каждую отдельную переменную 
link, source, name_news, time  = requests_to_link_ya(mainlink_ya, headers)

In [32]:
for name, link_1, time_1, source_1 in itertools.zip_longest(name_news,link,time,source):
    try:
        news_yandex_ru.insert_one({'_id':name,
                                'time':time_1,
                                'source':source_1,
                                'link':link_1})
    except:
        continue

In [33]:
for i in news_yandex_ru.find({}):
    pprint(i)

{'_id': ObjectId('5f05c5ce60dbad26b449d174'),
 'link': '/news/story/Minprosveshheniya_izmenit_format_lineek_1_sentyabrya_v_shkolakh--fa5589cd1dd221a41572bd356c4fbfc5?lr=213&lang=ru&stid=s-mJb-AwivSjOrGaQCTz&persistent_id=104884729&rubric=index&from=index',
 'name_news': 'Минпросвещения изменит формат линеек 1 сентября в школах',
 'source': 'Известия ',
 'time': '15:51'}
{'_id': ObjectId('5f05c5ce60dbad26b449d175'),
 'link': '/news/story/Medvedev_stanet_pozhiznennym_senatorom_posle_prinyatiya_zakona--d4b6a4e50ba26f8e22ccd84f8634e37b?lr=213&lang=ru&stid=PZGuvUp8XdK_Ii82kM5T&persistent_id=105022615&rubric=index&from=index',
 'name_news': 'Медведев станет пожизненным сенатором после принятия закона',
 'source': 'РИА Новости ',
 'time': '15:50'}
{'_id': ObjectId('5f05c5ce60dbad26b449d176'),
 'link': '/news/story/Mishustin_zayavil_ob_ugroze_dostizheniyam_v_borbe_s_bednostyu_v_Rossii--67d3a8790f6064d7ae08e6a77b560068?lr=213&lang=ru&stid=0dUNL-nGTo-As0MRWxQV&persistent_id=105040681&rubric=inde

In [37]:
from lxml import html
import requests

templates = [{"target": "LentaRu",
              "host": "https://lenta.ru",
              "title": "//div[@class='item']/a/text()",
              "url": "//div[@class='item']/a/@href",
              "pub": "//div[@class='item']/a/time[@class='g-time']/@datetime",
              "source": "LentaRu"
              },
             {"target": "Yandex",
              "host": "https://yandex.ru/news",
              "title": "//div[@class='story__topic']/h2/a/text()",
              "url": "//div[@class = 'page-content']//h2/a/@href",
              "pub": "//div[@class='mg-grid__row mg-grid__row_gap_8 news-top-stories news-app__top']//span[@class='mg-card-source__time']/text()",
              "source": "Заглушка"
              },
             {"target": "MailRU",
              "host": "https://news.mail.ru",
              "title": "//div[@class='daynews__item']//span[@class='photo__title photo__title_new photo__title_new_hidden js-topnews__notification']/text()",
              "url": "//div[@class='daynews__item']//a/@href",
              "pub": "//span[@class='note__text breadcrumbs__text js-ago']/@datatime",
              "source": "//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']/text()"
              }
             ]

# заголовок
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 '
                        'Safari/537.36', 'Accept': '*/*'}


# отправляем GET-запрос и получаем ответ от сервера для дальнейшего анализа
def get_page(url):
    response = requests.get(url, headers=header)
    return response


def get_content(response, params: dict):
    dom = html.fromstring(response.text)
    list_news = []
    title = dom.xpath(params['title'])
    url = dom.xpath(params['url'])
    url = add_domain(url, params['host'])

    pub = dom.xpath(params['pub'])
    source = dom.xpath(params['source'])
    list_news.append({'source': source, 'title': title, 'url': url, 'pub': pub})
    return list_news


def add_domain(list_url: list, domain: str):
    for index, item in enumerate(list_url):
        if item.find('http'):
            list_url[index] = domain + list_url[index]
        else:
            pass
    return list_url


def yandex_fix():
    pass


def mail_fix():
    pass


response = get_page(templates[1]['host'])
hot_news = get_content(response, templates[1])
# pprint(hot_news)
for i in hot_news[0]['title']:
    print(i)

В Кремле прокомментировали ситуацию с задержанием россиян в Белоруссии
Жена рэпера Энди Картрайта расчленила мертвого мужа
Геркус сообщил о переходе Кокорина в «Спартак»
В Минздраве назвали срок подготовки новых правил медосмотра водителей
Кравчук возглавит делегацию Киева в контактной группе по Донбассу
Жители еще семи домов в Коньково начали переселение по реновации
Более 37 тыс. пассажиров в Москве оштрафовали за отсутствие масок
Циклонический вихрь принесет непогоду в Москву в начале августа
Собянин объявил начало приема заявок от НКО на выдачу грантов
Число новых случаев Covid‑19 уменьшилось на 9% за месяц в Москве
Наталия Орейро примерила образ из «Дикого ангела» 20 лет спустя
Александра Черно родила сына весом 3640 г и ростом 55 см
Среди задержанных в Белоруссии боевиков есть члены батальона Прилепина
«Норникель» во втором квартале нарастил выпуск никеля на 8%
В НАСА объявили о запуске нового марсохода к Красной планете
Володин дал поручение проверить депутатов на двойное гражда